# Sailco problem
Sailco manufactures sailboats. During the next 4 months the company must
meet the following demands for their sailboats:

|Month | 1 | 2 | 3 | 4 |
|------|---|---|---|---|
|Number of boats | 40 | 60 | 70 | 25 |

At the beginning of Month 1, Sailco has 10 boats in inventory. Each month
it must determine how many boats to produce. During any month, Sailco can
produce up to 40 boats with regular labor and an unlimited number of boats
with overtime labor. Boats produced with regular labor cost \$400 each to
produce, while boats produced with overtime labor cost \$450 each. It costs \$20 to hold a boat in inventory from one month to the next. Find the
production and inventory schedule that minimizes the cost of meeting the
next 4 months' demands.

In [72]:
A =
[1   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0;
-1   1  1  0  0  0  0  0  0 -1 -1  0  0  0  0  0  0;
 0  -1  0  1  1  0  0  0  0  0  0 -1 -1  0  0  0  0;
 0   0  0 -1  0  1  1  0  0  0  0  0  0 -1 -1  0  0;
 0   0  0  0  0 -1  0  1  1  0  0  0  0  0  0 -1 -1;
 0   0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0; 
 0   0 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0; 
 0   0  0  0 -1  0  0  0  0  0  0  0  0  0  0  0  0; 
 0   0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0  0; 
 0   0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0;
 0   0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0;
 0   0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0;
 0   0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0;
 0   0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0;
 0   0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0;
 0   0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0;
 0   0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0;
 0   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1]

b = [10 0 0 0 0]
demand = [40 60 70 25]
numnodes = size(A,1);
numedges = size(A,2);

heldedges = 2:2:8;

In [73]:
using JuMP, HiGHS

m = Model(HiGHS.Optimizer)

#Boats produced with regular labor
@variable(m, 0 <= x[1:4] <= 40)
#Boats produced with overtime labor
@variable(m, y[1:4] >= 0)

#Edges in graph
@variable(m, edges[1:numedges] >= 0)

@variable(m, finalinventory >= 0)

@constraint(m, flow, A*edges .==  [b'; finalinventory; -demand'; x; y])

@objective(m, Min, 400*sum(x) + 450*sum(y)+ 20*(sum(edges[i] for i in heldedges)+edges[1]))

print(m)


In [74]:
optimize!(m)

Presolving model
4 rows, 10 cols, 13 nonzeros
3 rows, 9 cols, 11 nonzeros
Presolve : Reductions: rows 3(-15); columns 9(-17); elements 11(-32)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
          4     7.6400000000e+04 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 4
Objective value     :  7.6400000000e+04
HiGHS run time      :          0.01


In [75]:
@show objective_value(m)

@show value.(x)
@show value.(y);

objective_value(m) = 76400.0
value.(x) = [40.0, 40.0, 40.0, 25.0]
value.(y) = [-0.0, 10.0, 30.0, -0.0]
